In [ ]:
!pip install transformers # installing tranformers

In [ ]:
# importing modules and packages
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertModel, BertTokenizer
import torch
import json
import progressbar
from transformers import BertTokenizer, TFBertModel
import torch
from transformers import AutoTokenizer, AutoModel

In [ ]:
# mounting drive in colab
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
file=open("/content/judgement_prediction_test.txt",'r') # path of the text file with test filenames
file_name=file.read()
list_of_file_names=file_name.split('\n') # list containing the names of the test files

In [ ]:
tag_doc=open("/content/tag.txt",'r') # path of the text file with filenames and its respective tags as dictionary
tag_doc_text=tag_doc.read()
list_tags_as_string=[]
tag_doc_text=eval(tag_doc_text) #converting the string to dictionary
for fl_name in tag_doc_text:
  str=''
  for tag in tag_doc_text[fl_name]:
    str+=tag+' '
  list_tags_as_string.append(str) # appending the tags as space separated string in a list

In [ ]:
# Loading the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")


# Load the Longformer model and tokenizer
'''model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerModel.from_pretrained(model_name)'''

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#model = BertModel.from_pretrained("bert-base-uncased")
#tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
#model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ILDC_single.csv') # path of the csv file containg the train and test set
train_set= df.query(" split=='train' ") #separting the train set
test_set= df.query(" split=='test' ") # separating the test set

In [ ]:
import numpy as np
def generate_np_files_for_training(text, tokenizer): # function to return the embedding of a text using above tokenizer and model
  all_input_ids, all_att_masks, all_labels = [], [], []
  toks = tokenizer.tokenize(text)
  if(len(toks) > 10000): # if the token size is more than 10000, drop the first 10000 tokens
      toks = toks[len(toks)-10000:]
  tokens = ['[CLS]'] + toks[-510:] + ['[SEP]'] # model uses 512 tokens
  input_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_tensor = torch.tensor(input_ids)
  # Get the model's output
  with torch.no_grad():
    output = model(input_tensor.unsqueeze(0))
  embedding = output.last_hidden_state[:, 0, :]
  embedded=embedding.tolist()
  return embedded[0] #returning the embedding as a list

In [ ]:
def embed(dataf): #function to return embedding of all text's in a dataframe
  embedded_list=[]
  for i in progressbar.progressbar(range(len(dataf['text']))):
    text = dataf['text'].iloc[i]
    k=generate_np_files_for_training(text, tokenizer) # calling the above method and passing the text and loaded tokenizer
    embedded_list.append(k)
  return embedded_list


In [ ]:
train_embedding=embed(train_set) #embedding of the whole train set
X=np.array(train_embedding)
np.save("/content/drive/MyDrive/experiment/inlegaltraining.npy", X) #saving the embedding of the train set as a binary file

In [ ]:
test_filename=test_set["name"].tolist() # names of all testfile
lst1,lst2,lst3=[],[],[]
for i in list_of_file_names:
  set,set1=[],[]
  if i in test_filename:
    set=generate_np_files_for_training(test_set.loc[5082+test_filename.index(i)].at["text"],tokenizer) # embedding of the text of the test file without appended tag
    set1=generate_np_files_for_training(test_set.loc[5082+test_filename.index(i)].at["text"]+list_tags_as_string[list_of_file_names.index(i)],tokenizer) #embedding of the text with appended tag
    lst2.append(int(test_set.loc[5082+test_filename.index(i)].at["label"])) # extracting the original label of the test files
    lst1.append(set)
    lst3.append(set1)
embedding_without_appended_tag=np.array(lst1)
embedding_appended_text=np.array(lst3)

In [ ]:
import numpy as np
train_embedding=np.load("/content/drive/MyDrive/experiment/inlegaltraining.npy",allow_pickle=True) #loading the binary file with embedding of the train set

In [ ]:
'''def Listing(a):
  l=[]
  for i in a:
    x=i.tolist()
    l.append(x)
  return l'''

In [ ]:
'''def padding_zeros(a):
  for i in a:
    i.extend([0]*(768))'''

In [ ]:
'''train=Listing(tr)
padding_zeros(train)
trn=np.array(train)
tst=np.array(lst1)
print(len(tst[0]))'''

In [ ]:
train_label = train_set['label'].to_numpy().astype('int') # labels of the train set
test_label=np.array(lst2)

In [ ]:
from sklearn.linear_model import LogisticRegression # importing Logistic Regression from sklearn
from sklearn.metrics import accuracy_score # importing accuracy_score
clf = LogisticRegression() # loading LR
clf.fit(train_embedding,train_label) # fitting LR with our train set embedding and labels
test_predicted_label_w_tag=clf.predict(embedding_appended_text) # predicted label of test set with appended tag at the end
test_predicted_label_wo_tag=clf.predict(embedding_without_appended_tag) #predicted labe of test set without appended tag
acc1=accuracy_score(test_label,test_predicted_label_w_tag) # accuracy calculation using original and predicted label
acc=accuracy_score(test_label,test_predicted_label_wo_tag)
print("Accuracy without appending tag:",acc)
print("Accuracy with appended tag:",acc1)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
clf=SVC(kernel="linear")
clf.fit(tr,trlabel)
t_label=clf.predict(tst)
t1_label=clf.predict(tst2)
acc=accuracy_score(tst1,t_label)
acc1=accuracy_score(tst1,t1_label)
print(acc,acc1)

In [ ]:
from sklearn.svm import SVC # importing Lofistic Regression from sklearn
from sklearn.metrics import accuracy_score # importing accuracy_score
clf = SVC(kernel="linear") # loading Linear SVC
clf.fit(train_embedding,train_label) # fitting LR with our train set embedding and labels
test_predicted_label_w_tag=clf.predict(embedding_appended_text) # predicted label of test set with appended tag at the end
test_predicted_label_wo_tag=clf.predict(embedding_without_appended_tag) #predicted labe of test set without appended tag
acc1=accuracy_score(test_label,test_predicted_label_w_tag) # accuracy calculation using original and predicted label
acc=accuracy_score(test_label,test_predicted_label_wo_tag)
print("Accuracy without appending tag:",acc)
print("Accuracy with appended tag:",acc1)